In [ ]:
# May 16th: Transformer 4.37.0 via pip + AutoGPTQ 0.8.0dev0 from source (git clone) works on Colab
!pip install transformers==4.37.0
###!pip3 install git+https://github.com/huggingface/transformers

!git clone https://github.com/AutoGPTQ/AutoGPTQ.git
%cd AutoGPTQ
!pip install -e .

# Upgrade relevant package

!pip install --upgrade trl peft accelerate bitsandbytes datasets optimum -q

# Verify the versions
import transformers
import auto_gptq

print("Transformers version:", transformers.__version__)
print("AutoGPTQ version:", auto_gptq.__version__)

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
import torch
from transformers import AutoTokenizer, MixtralForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig, GPTQConfig
from datasets import load_dataset
import pandas as pd
import logging
import os
from pathlib import Path
from typing import Optional, Tuple
from peft import LoraConfig, PeftConfig, PeftModel, prepare_model_for_kbit_training, LoraConfig, get_peft_model

In [ ]:
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
# (OPTIONAL) Load dataset from HuggingFace

dataset = load_dataset('gbharti/finance-alpaca')

# Split the dataset into train and test sets
train_test_split = dataset['train'].train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Further split the train dataset into train and validation sets
train_val_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_val_split['train']
eval_dataset = train_val_split['test']

pretrained_model_name_or_path = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"

In [ ]:
# (OPTIONAL) Read prompt data from excel file

import os
import pandas
import json
from datasets import Dataset, load_dataset

finetune_prompt_train = []  # Initialize as a list
finetune_prompt_test = []
df = pandas.read_excel("/content/gen_sample_20.xlsx")
'''for i in range(19):
    prompt = df.loc[i, '(模擬律師output)輸入內容']
    finetune_answer = df.loc[i, '(起訴狀answer)H3']
    finetune_temp = f"""<s>[INST]{prompt}[/INST]{finetune_answer}</s>"""
    if i < 16:
      finetune_prompt_train.append(finetune_temp)  # Append to the list
    else:
      finetune_prompt_test.append(finetune_temp)  # Append to the list
print(finetune_prompt_train[0])
print(finetune_prompt_test[0])'''

def create_text_row(instruction, output):
  #暫時捨棄了input以減少token數
    text_row = f"""<s>[INST] {instruction} [/INST] \n {output} </s>"""
    return text_row

# interate over all the rows formate the dataset and store it in a jsonl file
def process_jsonl_file(output_file_path):
    with open(output_file_path, "w") as output_jsonl_file:
        for i in range(20):
            instruction = df.loc[i, '(模擬律師output)輸入內容']
            output = df.loc[i, '(起訴狀answer)H3']
            json_object = {
                "text": create_text_row(instruction, output),
                "instruction": instruction,
                "input": "", # 暫時捨棄了input以減少token數
                "output": output
            }
            output_jsonl_file.write(json.dumps(json_object, ensure_ascii=False) + "\n")

# Provide the path where you want to save the formatted dataset
process_jsonl_file("./train_dataset.jsonl")
dataset = load_dataset('json', data_files='./train_dataset.jsonl' , split='train')

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

# Further split the train dataset into train and validation sets
train_val_split = train_dataset.train_test_split(test_size=0.1)
train_dataset = train_val_split['train']
eval_dataset = train_val_split['test']


##############

###pretrained_model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"
pretrained_model_name_or_path = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ"
revision="gptq-3bit--1g-actorder_True"	

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=2048,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

def format_input_data_to_build_model_prompt(data_point):
        instruction = str(data_point['instruction'])
        input_query = str(data_point['input'])
        response = str(data_point['output'])

        if len(input_query.strip()) == 0:
            full_prompt_for_model = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction} \n\n### Input:\n{input_query}\n\n### Response:\n{response}"""

        else:
            full_prompt_for_model = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n{response}"""
        return tokenize(full_prompt_for_model)

def build_qlora_model(
    ###pretrained_model_name_or_path: str = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
    pretrained_model_name_or_path: str = "TheBloke/Mixtral-8x7B-Instruct-v0.1-GPTQ",
    gradient_checkpointing: bool = True,
    cache_dir: Optional[Path] = None,
) -> Tuple[MixtralForCausalLM, AutoTokenizer, PeftConfig]:
    """
    Args:
        pretrained_model_name_or_path (str): The name or path of the pretrained model to use.
        gradient_checkpointing (bool): Whether to use gradient checkpointing or not.
        cache_dir (Optional[Path]): The directory to cache the model in.

    Returns:
        Tuple[MixtralForCausalLM, AutoTokenizer]: A tuple containing the built model and tokenizer.
    """

    # Disable bnb_config if using any GPTQ model (Can't quantize an already quantized model)

    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_use_double_quant=True,
    #     bnb_4bit_compute_dtype=torch.bfloat16
    # )

    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path,
        padding_side="left",
        add_eos_token=True,
        add_bos_token=True,
    )
    tokenizer.pad_token = tokenizer.eos_token

    quantization_config_loading = GPTQConfig(bits=3, use_exllama=False, tokenizer=tokenizer)

    # Disable quantization_config param if using GPTQ models

    model = MixtralForCausalLM.from_pretrained(
        pretrained_model_name_or_path,
        # quantization_config=bnb_config,
        quantization_config=quantization_config_loading,
        device_map="auto",
        cache_dir=str(cache_dir) if cache_dir else None,
    )

    # Disable tensor parallelism
    model.config.pretraining_tp = 1

    if gradient_checkpointing:
        model.gradient_checkpointing_enable()
        model.config.use_cache = (
            False  # Gradient checkpointing is not compatible with caching.
        )
    else:
        model.gradient_checkpointing_disable()
        model.config.use_cache = True  # It is good practice to enable caching when using the model for inference.

    return model, tokenizer

In [ ]:
model, tokenizer = build_qlora_model(pretrained_model_name_or_path)
model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

print_trainable_parameters(model)

# Output format:
# trainable params: 6815744 || all params: 269225984 || trainable%: 2.5316070532033046

# Apply the accelerator. (optional)
model = accelerator.prepare_model(model)

tokenized_train_dataset = train_dataset.map(format_input_data_to_build_model_prompt)
tokenized_val_dataset = eval_dataset.map(format_input_data_to_build_model_prompt)

In [ ]:
# Grabs a single data point from our testset to see how the base model does on it.

print("Instruction Sentence: " + test_dataset[1]['instruction'])
print("Output: " + test_dataset[1]['output'] + "\n")

# Prompt's template for the Instruct model is defined as follows:
#<s> [INST] Instruction [/INST] Model answer</s> [INST] Follow-up instruction [/INST]

In [ ]:
eval_prompt = """Given an instruction sentence construct the output.

### Instruction sentence:
Generate a sentence that describes the main idea behind a stock market crash.


### Output


"""

In [ ]:
#Now, to start our fine-tuning, we have to apply some preprocessing to the model to prepare it for training. For that use the prepare_model_for_kbit_training method from PEFT.
# Apply the accelerator. (prepare_model - Prepares a PyTorch model for training in any distributed setup.)
model = accelerator.prepare_model(model)

# Re-init the tokenizer so it doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    add_bos_token=True,
)

device = "cuda"
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to(device)

In [ ]:
model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=128)[0], skip_special_tokens=True))

In [ ]:
print(model)

In [ ]:
# Check Torch, CUDA version

import torch


print("PyTorch version:", torch.__version__)

if torch.cuda.is_available():
    print("CUDA is available")
    print("CUDA version:", torch.version.cuda)
else:
    print("CUDA is not available")

if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True
    print("Multiple CUDA detected, parallel model enabled.")
else:
    print("A single CUDA detected.")

In [ ]:
# Training
import transformers
from datetime import datetime

# Enable CUDA launch blocking for detailed error messages
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

project = "Mixtral-alpaca-finance-finetune"
base_model_name = "mixtral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_checkpointing=True,
        gradient_accumulation_steps=2, # reduced from 4 for testing
        max_steps=1000,
        learning_rate=2.5e-5,
        logging_steps=25,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",                   # Directory for storing logs
        save_strategy="steps",                  # Save the model checkpoint every logging step
        save_steps=50,                          # Save checkpoints every 50 steps
        evaluation_strategy="steps",            # Evaluate the model every logging step
        eval_steps=50,                          # Evaluate and save checkpoints every 50 steps
        do_eval=True,                           # Perform evaluation at the end of training
        # report_to="wandb",                    # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}" # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Re-enable for inference!
trainer.train()

In [ ]:
# Post-training Inference
from transformers import pipeline, logging

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "你的繁體中文能力如何? [/INST]我可以進行流暢的繁體中文對話，請儘管發問! [INST] 任務內容：從判決書的內容反推律師在撰寫起訴狀的時候的會使用的敘述。\
最下面會有判決書的內容，請你從判決書內容生成模擬律師如果要生成起訴狀的話，會有哪些必要的語句，盡量要口語化和簡單。\
我要改寫成三個大段落：\
一、 事故發生緣由\
二、 被告受傷情形\
三、 包含請求賠償的事實根據\
需要保留內容：重點事項要維持一樣的，包括如果原本在判決書中有條文的話，雖然不口語但是也要保留；如果是被告和原告的互動描述也要保留；受傷情形、事故發生緣由、包含請求賠償的事實根據保持原有的詳細程度；要保留相關的數字，像是住院長度，不能工作的長度，也都要保留；地址、姓名、車牌等訊息也要保留，不要自己新創內容；如果原文有引用判決的時候，必須要保留把判決的年度跟號碼寫出來。\
需要注意的內容：你是要模擬律師，所以主詞還是都要是'被告'或是'原告'，不要用'我'；'不要'把賠償金額加總起來或是計算，就單純保持原樣即可。\
內容要求：非常的口語化和簡單。\
以下為判決書內容： [/INST]請告訴我判決書內容以生成起訴狀 [INST] 一、事故發生緣由：\
被告在民國109年4月2日晚上7點40分左右，開著車牌號碼是0000-00的自用小客車，從臺北市文山區木新路3段的西邊往東邊開。開到木新路3段和興隆路4段的路口時，被告想要左轉到興隆路4段的北向車道。按照交通規則，被告應該要注意車子前面的狀況，隨時採取必要的安全措施，也要特別注意行人穿越道上有沒有行人。如果有行人要過馬路，不管有沒有交通指揮人員或號誌，被告都應該要停車讓行人先走。而且當時的天氣、路況和能見度都很好，被告沒有理由說他不能注意。但是被告就是疏忽大意，沒有發現原告正要從西邊往東邊走過行人穿越道，就這樣直接開過去，撞到了原告。原告因此受傷，傷勢包括頭部外傷、頸部拉傷和挫傷、右手肘和右手挫傷、下背部挫傷、頸部扭傷等等。\
\
二、被告受傷情形：\
原告在車禍發生當天，也就是109年4月2日，先被送到臺北市立萬芳醫院急診。後來又繼續在萬芳醫院門診追蹤治療，還去了吉辰中醫診所、春林復健科診所、臺北市立聯合醫院松德院區、中山醫療社團法人中山醫院、臺北榮民總醫院、尹書田醫療財團法人書田泌尿科眼科診所等醫院復健。原告為了生活上的需要，還花錢買了一些醫療用品。這些都有診斷證明書和醫療收據可以證明。原告請求被告賠償的醫療費用和增加生活上需要的費用，總共是13萬元，這是依照民法第193條第1項的規定。\
\
三、請求賠償的事實根據：\
在車禍發生之前，原告在基邑園藝工程有限公司上班，每個月薪水是45,000元。從車禍發生後到110年1月，原告已經有9個月不能工作，薪資損失本來應該有40萬5,000元，但是原告只請求20萬元而已。而且原告因為車禍受傷，身心都受到創傷，還得了重度憂鬱症和創傷後壓力症候群。車禍的後遺症讓原告沒辦法久站或久坐，到現在都還在休養當中，沒有辦法工作。此外，原告在這次車禍受傷以後，每次走在行人穿越道上都會擔心再次被車撞飛，精神一直處在緊張不安的狀態。而且到現在傷勢都還沒完全康復，要繼續門診追蹤和復健，嚴重影響了生活作息。原告的精神上受到很大的痛苦，心理壓力也很大，加上原告還要一個人扶養兩個還在讀書的小孩，所以原告請求被告賠償50萬元的精神慰撫金。"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4096)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
# Post-training Inference
from transformers import pipeline, logging

# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = """你的繁體中文能力如何? [/INST]我可以進行流暢的繁體中文對話，請儘管發問! [INST] 你是一個中華民國的民法專長的律師，你主要是要撰寫交通事故的民事起訴狀，並且只針對慰撫金部分的案件作撰寫。慰撫金只會用到民法的內容。
以下是民事起訴狀的結構：
一、(這裡要填入詳細的事實緣由，要用法律用語)
二、(這裡先'引用法條'，因為我們要生成的起訴狀是限定在有慰撫金的民事的交通事故，基本上都會用到第184條，請你注意。請先閱讀要改寫的內容，如果有請求看護費用，引用第193條會比較合理，如果沒有要賠償看護費用的話，就不須引用第193條。如果在2個以上的被告需要共同賠償的時候，大多會引用第185條第1項。接著會開始書寫連帶賠償。)
 |---(一)(這裡填入相關的賠償大項，以及說明，說明要很詳細。撰寫金額的時候統一使用'位數逗號的數字'；已支出費用跟未來發生的費用不可以寫在同一大項中，要分開敘述。)
 |         |---1.(這裡填入大項中的細項賠償，以及說明，說明要很詳細。撰寫金額的時候統一使用'位數逗號的數字'。)
 |                 |----(1)(這裡填入細項賠償中的說明，說明要很詳細。撰寫金額的時候統一使用'位數逗號的數字'。)
 |
 |---(六) (最後一個是要結尾這一大段的，要以'綜上所陳'開頭對這一段賠償做結尾；要把所有的賠償金額都相加起來，請務必要加總正確，撰寫金額的時候統一使用'位數逗號的數字'。)
請將以下律師口語輸入的內容根據民事起訴狀的格式改寫出來。文章風格、格式和階層格式'務必'要與範本和我要的結構一樣；不要根據以下的格式；不要新創內容，完全依照以下我提供的事實做改寫：
"一、事故發生緣由:
被告的機車駕照因為酒駕被註銷了,但是在民國110年8月18日晚上9點26分左右,他還是騎著車牌號碼是000-0000的重型機車,在新北市板橋區長江路1段由南往北往民生路的方向騎。雖然當時是晚上,但是路上有路燈、天氣晴朗、路面是乾的也沒有坑洞,視線也很好,被告應該要注意迴轉時要打方向燈,並且停下來看清楚有沒有車輛來往,可是被告卻疏忽了,沒有打方向燈就直接從路邊起駛要左轉。

二、原告受傷情形:
就在被告要左轉的時候,原告騎著車牌號碼是000-0000的重型機車從後面同向騎來,因為來不及閃,所以兩台機車就撞在一起。原告跟機車一起摔在地上,原告的左膝挫傷還合併十字韌帶撕裂性骨折,左肩也挫傷和擦傷。原告總共動了兩次手術,每次手術後都需要專人照顧一個月。受傷、開刀到術後休養加起來超過7、8個月不能工作。

三、請求賠償的事實根據:
原告請求被告賠償的項目包括:醫療費用171,170元、護具費用9,500元、就醫回診的交通費5,000元、看護費用90,000元、不能工作的薪資損失630,000元、維修受損機車的費用31,800元,以及精神慰撫金300,000元。原告在事故前受僱於力拔山工程有限公司當泥作師傅,每個月的薪水是105,000元。因為這起事故,原告的機車壞了需要修,身體也受傷需要手術和休養,不但身體感到疼痛,生活作息也受到很大影響,精神上也承受了相當大的痛苦。" """
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4096)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

**Save Model**

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

In [ ]:
# Reload model in FP16 and merge it with LoRA weights

base_model = MixtralForCausalLM.from_pretrained(
    model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device = device
)

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     low_cpu_mem_usage=True,
#     return_dict=True,
#     torch_dtype=torch.float16,
#     device_map={"": 0},
# )

# Fine-tuned model name
new_model = "mixtral-8x7b-finetune"

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
test_prompt = f"""一、事故發生緣由：
被告在民國109年4月2日晚上7點40分左右，開著車牌號碼是0000-00的自用小客車，從臺北市文山區木新路3段的西邊往東邊開。開到木新路3段和興隆路4段的路口時，被告想要左轉到興隆路4段的北向車道。按照交通規則，被告應該要注意車子前面的狀況，隨時採取必要的安全措施，也要特別注意行人穿越道上有沒有行人。如果有行人要過馬路，不管有沒有交通指揮人員或號誌，被告都應該要停車讓行人先走。而且當時的天氣、路況和能見度都很好，被告沒有理由說他不能注意。但是被告就是疏忽大意，沒有發現原告正要從西邊往東邊走過行人穿越道，就這樣直接開過去，撞到了原告。原告因此受傷，傷勢包括頭部外傷、頸部拉傷和挫傷、右手肘和右手挫傷、下背部挫傷、頸部扭傷等等。

二、被告受傷情形：
原告在車禍發生當天，也就是109年4月2日，先被送到臺北市立萬芳醫院急診。後來又繼續在萬芳醫院門診追蹤治療，還去了吉辰中醫診所、春林復健科診所、臺北市立聯合醫院松德院區、中山醫療社團法人中山醫院、臺北榮民總醫院、尹書田醫療財團法人書田泌尿科眼科診所等醫院復健。原告為了生活上的需要，還花錢買了一些醫療用品。這些都有診斷證明書和醫療收據可以證明。原告請求被告賠償的醫療費用和增加生活上需要的費用，總共是13萬元，這是依照民法第193條第1項的規定。

三、請求賠償的事實根據：
在車禍發生之前，原告在基邑園藝工程有限公司上班，每個月薪水是45,000元。從車禍發生後到110年1月，原告已經有9個月不能工作，薪資損失本來應該有40萬5,000元，但是原告只請求20萬元而已。而且原告因為車禍受傷，身心都受到創傷，還得了重度憂鬱症和創傷後壓力症候群。車禍的後遺症讓原告沒辦法久站或久坐，到現在都還在休養當中，沒有辦法工作。此外，原告在這次車禍受傷以後，每次走在行人穿越道上都會擔心再次被車撞飛，精神一直處在緊張不安的狀態。而且到現在傷勢都還沒完全康復，要繼續門診追蹤和復健，嚴重影響了生活作息。原告的精神上受到很大的痛苦，心理壓力也很大，加上原告還要一個人扶養兩個還在讀書的小孩，所以原告請求被告賠償50萬元的精神慰撫金。"""
messages = [
    {"role": "user", "content": "你的繁體中文能力如何?"},
    {"role": "assistant", "content": "我可以進行流暢的繁體中文對話，請儘管發問!"},
    {"role": "user", "content": f"""任務內容：從判決書的內容反推律師在撰寫起訴狀的時候的會使用的敘述。
最下面會有判決書的內容，請你從判決書內容生成模擬律師如果要生成起訴狀的話，會有哪些必要的語句，盡量要口語化和簡單。
我要改寫成三個大段落：
一、 事故發生緣由
二、 被告受傷情形
三、 包含請求賠償的事實根據
- 需要保留內容：重點事項要維持一樣的，包括如果原本在判決書中有條文的話，雖然不口語但是也要保留；如果是被告和原告的互動描述也要保留；受傷情形、事故發生緣由、包含請求賠償的事實根據保持原有的詳細程度；要保留相關的數字，像是住院長度，不能工作的長度，也都要保留；地址、姓名、車牌等訊息也要保留，不要自己新創內容；如果原文有引用判決的時候，必須要保留把判決的年度跟號碼寫出來。
- 需要注意的內容：你是要模擬律師，所以主詞還是都要是'被告'或是'原告'，不要用'我'；'不要'把賠償金額加總起來或是計算，就單純保持原樣即可。
- 內容要求：非常的口語化和簡單。
以下為判決書內容："""},
    {"role": "assistant", "content": "請告訴我判決書內容以生成起訴狀"},
    {"role": "user", "content": test_prompt}
]
input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
outputs = model.generate(input_ids, max_new_tokens=4096)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# (OPTIONAL)
# Upload the finetuned model to HuggingFace

import locale
locale.getpreferredencoding = lambda: "UTF-8"

!huggingface-cli login

model.push_to_hub("Llamarider222/Mixtral-8x7b-Instruct-GPTQ", check_pr=True)

tokenizer.push_to_hub("Llamarider222/Mixtral-8x7b-Instruct-GPTQ",check_pr=True)

config.push_to_hub("Llamarider222/Mixtral-8x7b-Instruct-GPTQ", check_pr=True)

In [ ]:
# Evaluate the trained model
import torch
from transformers import AutoTokenizer, MixtralForCausalLM, BitsAndBytesConfig

pretrained_model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

base_model = MixtralForCausalLM.from_pretrained(
    pretrained_model_name_or_path,  # Mixtral, same as before
    # quantization_config=bnb_config,  # Same quantization config as before, but commented out as its a GPTQ model (which is already quantized )
    quantization_config=quantization_config_loading,
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    add_bos_token=True,
    trust_remote_code=True,
)

**Load Model**

In [ ]:
# Load model from HuggingFace

from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-finetune-alpaca-GPTQ/checkpoint-500")

# Here, "mistral-finetune-alpaca-GPTQ/checkpoint-500" is the adapter name

In [ ]:
eval_prompt = """"Given an instruction sentence construct the output.

### Instruction sentence:
Generate a sentence that describes the main idea behind a stock market crash.


### Output


"""

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=50)[0], skip_special_tokens=True))